<a href="https://colab.research.google.com/github/bhavikngala/object_detection_in_point_clouds/blob/dev/pixor_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /gdrive


### Setup

In [3]:
%cd /content
!git clone --branch doogal https://<USERNAME>:<PASSWORD>@github.com/bhavikngala/object_detection_in_point_clouds.git
!mv object_detection_in_point_clouds code
%cd ./code
!git pull origin doogal

/content
Cloning into 'object_detection_in_point_clouds'...
remote: Enumerating objects: 135, done.
remote: Counting objects: 100% (135/135), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 1895 (delta 86), reused 76 (delta 36), pack-reused 1760
Receiving objects: 100% (1895/1895), 6.10 MiB | 21.77 MiB/s, done.
Resolving deltas: 100% (1258/1258), done.
/content/code
From https://github.com/bhavikngala/object_detection_in_point_clouds
 * branch            doogal     -> FETCH_HEAD
Already up to date.


In [4]:
!mkdir -p models
!mkdir -p loss
!pwd

/content/code


### Custom trainer

In [0]:
import torch
from torch.utils.data import DataLoader

import os
import traceback
import argparse
import numpy as np
try:
  from tensorboardX import SummaryWriter
  log = True
except:
  log = False

import datautils.dataloader_v3 as dataloader
import config as cnf
import misc
import networks.networks as networks
import model_groomers as mg
import lossUtils as lu

In [0]:
class CustomGroomer(mg.ModelTrainer):
  def __init__(self, log, logDir, modelFilename, **kwargs):
    if log: self.writer = SummaryWriter(logDir)
    self.iter = 0
    self.logDir = logDir
    self.modelFilename = modelFilename
    self.clip_value = kwargs['clip_value']
    self.accumulationSteps = kwargs['accumulationSteps']
    self.lrRange2 = kwargs['lrRange2']
    self.momentumRange2 = kwargs['momentumRange2']

    
  def train(self, device, log=False):
    if self.loader is None:
      print('data loader is undefined')
      quit()
      
    self.model.train()

    subBatchCounter = 0
    for epoch in range(1, self.epochs+1):
      epochValues = []

      self.scheduler.step()

      batch_size = self.loader.batch_size
      self.model.zero_grad()
      for batchId, data in enumerate(self.loader):
        lidar, targetClass, targetLoc, filenames = data

        # ignore last batch
        if lidar.size(0) != batch_size:
          continue

        lidar = lidar.cuda(device, non_blocking=True)
        targetClass = [c.contiguous().cuda(device, non_blocking=True) for c in targetClass]
        targetLoc = [loc.contiguous().cuda(device, non_blocking=True) for loc in targetLoc]

        predictedClass, predictedLoc = self.model(lidar)

        claLoss, locLoss, trainLoss, posClaLoss, negClaLoss, meanConfidence, overallMeanConfidence, numPosSamples, numNegSamples \
         = self.lossFunction(predictedClass, predictedLoc, targetClass, targetLoc, self.alpha1, self.beta1)

        if trainLoss is not None:
          trainLoss = trainLoss/self.accumulationSteps
          trainLoss.backward()
          subBatchCounter += 1

        if subBatchCounter==self.accumulationSteps:
          self.optim.step()
          self.model.zero_grad()
          subBatchCounter = 0

    
        epochValues.append((claLoss, locLoss, trainLoss, posClaLoss, negClaLoss, meanConfidence, overallMeanConfidence, numPosSamples, numNegSamples))
      
      self.epochLogger(epochValues, epoch, log)
      self.saveModel(self.modelFilename)

      
  def val(self):
    pass


  def epochLogger(self, epochValues, epoch, log=False):
    pC, nC, locL, mC, mPT, nP, nN = 0, 0, 0, 0, 0, 0, 0
    for i in range(len(epochValues)):
      claLoss, locLoss, trainLoss, posClaLoss, negClaLoss, meanConfidence, overallMeanConfidence, numPosSamples, numNegSamples = \
        epochValues[i]
      if posClaLoss is not None:
        pC = pC + posClaLoss*numPosSamples
        locL = locL + locLoss*numPosSamples
        mC = mC + meanConfidence*numPosSamples
        nC = nC + negClaLoss*numNegSamples
        mPT = mPT + overallMeanConfidence*(numPosSamples+numNegSamples)
        nP += numPosSamples
        nN += numNegSamples
        
    pC /= nP
    nC /= nN
    locL /= nP
    mC /= nP
    mPT /= (nP+nN)

    '''
    self.writer.add_scalar('train/epoch_classification_loss', pC+nC, epoch)
    self.writer.add_scalar('train/epoch_pos_classification_loss', pC, epoch)
    self.writer.add_scalar('train/epoch_neg_classification_loss', nC, epoch)
    self.writer.add_scalar('train/epoch_localization_loss', locL, epoch)
    self.writer.add_scalar('train/epoch_train_loss', pC+nC+locL, epoch)
    self.writer.add_scalar('train/epoch_mean_pos_sample_confidence', mC, epoch)
    self.writer.add_scalar('train/epoch_mean_pt', mPT, epoch)
    '''
    cl = pC+nC
    tl = cl+locL
    print(f'epoch: {epoch} | CL: {cl} | PC: {pC} | NC: {nC} | LL: {locL} | TL: {tl} | MPC: {mC} | MC: {mPT}')
      

  def setLoaders(self):
    pass

In [0]:
def main(args):
  # data loaders
  trainLoader = DataLoader(
    dataloader.KittiDataset(cnf, args, 'train'),
    batch_size = cnf.batchSize, shuffle=True, num_workers=3,
    collate_fn=dataloader.customCollateFunction, pin_memory=True
  )

  # define model
  model = networks.PointCloudDetector2(
    cnf.res_block_layers,
    cnf.up_sample_layers,
    cnf.deconv)

  modelTrainer = CustomGroomer(False, cnf.logDir, args.model_file,
    clip_value=cnf.clip_value,
    accumulationSteps=cnf.accumulationSteps,
    lrRange2 = cnf.lrRange2,
    momentumRange2 = cnf.momentumRange2)
  modelTrainer.setDataloader(trainLoader)
  modelTrainer.setEpochs(cnf.epochs)
  modelTrainer.setModel(model)
  modelTrainer.setDataParallel(args.multi_gpu)
  modelTrainer.copyModel(cnf.device)
  modelTrainer.setOptimizer('sgd', cnf.slr, cnf.momentum, cnf.decay)
  modelTrainer.setLRScheduler(cnf.lrDecay, cnf.milestones)

  if False and os.path.isfile(args.model_file):
    modelTrainer.loadModel(args.model_file)

  modelTrainer.setLossFunction(lu.computeLoss, cnf.alpha1, cnf.beta1)
  modelTrainer.train(cnf.device)

In [0]:
MODEL_FILE = './models/p1.pth'
cnf.rootDir = '/gdrive/My Drive/KITTI_BEV'
cnf.calTrain = '/gdrive/My Drive/KITTI_BEV/training/calib'
cnf.slr = 1e-2
cnf.milestones = [20, 30]
cnf.epochs = 40
print(cnf.device)

parser = argparse.ArgumentParser(description='Train network')
parser.add_argument('-f', '--model-file', default=None, help='used to set different model file name other than default one')
parser.add_argument('-m', '--multi-gpu', action='store_true', help='use multiple gpus?')
parser.add_argument('--full-train-set', action='store_true', help='use entire training set or split into train val set')
parser.add_argument('--plot-img', action='store_true', help='plot on img; used in validateNetwork.py')
args = parser.parse_args()
args.model_file = MODEL_FILE

try:
  main(args)
except Exception as e:
  with open('./error.txt', 'w') as f:
    print(traceback.format_exc())
    f.write(traceback.format_exc())

cuda


In [0]:
!tail ./../data/training/calib/000000.txt